## Hyper - tuning - RF model

The feature space, described below, and the RandomForrest classifier gives us the best validation AUC out of all other models. We will now use `BayesSearchCV` to hyper-tune the classifier on the feature space.

Engineered two different types of features,

1. n_gram similarity between each pair of questions
2. min/max/avg distance between words in a single question. Currently using the following metrics,
  * euclidean
  * cosine
  * city block or manhattan
  
**Pipeline**
1. Stack questions
2. Clean questions - now lower cases all words to better lemmatize proper nouns
3. UNION
    1. n_gram similarity
    2. min/max/avg distance
4. Lemmatize questions
5. UNION
    1. n_gram similarity
    2. min/max/avg distances
6. UNION together both sets of features
7. Random Forrest

In [1]:
# data manipulation
import utils
import pandas as pd
import numpy as np

# modeling
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

# parameter search
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

## Load data

In [3]:
X_train = utils.load('X_train')
y_train = utils.load('y_train')
model_name = 'rf_hypertune'

## Text transformation and Feature Engineer pipes

In [9]:
# text transformation pipes
clean_text = Pipeline(
    [
        ('stack', FunctionTransformer(utils.stack_questions, validate=False)),
        ('clean', FunctionTransformer(utils.clean_questions, validate=False))

    ]
)

lemma_text = Pipeline(
    [
        ('lemma', FunctionTransformer(utils.apply_lemma, validate=False))
    ]
)

# feature engineering pipes
single_question_pipe = Pipeline(
    [
        ('dist', FunctionTransformer(utils.add_min_max_avg_distance_features, validate=False)),
        ('unstack', FunctionTransformer(utils.unstack_questions, validate=False))
    ]
)

pair_question_pipe = Pipeline(
    [
        ('ngram_sim', FunctionTransformer(utils.calc_ngram_similarity, kw_args={'n_grams':[1, 2, 3]}, validate=False))
    ]
)

# build features on the cleaned text only
clean_text_features = Pipeline(
    [
        ('clean', clean_text),
        ('feats', FeatureUnion(
            [
                ('pair', pair_question_pipe),
                ('single', single_question_pipe)
            ]
        ))
    ]
)

# build features on the cleanned and lemmatized text features
lemma_text_features = Pipeline(
    [
        ('clean', clean_text),
        ('lemma', lemma_text),
        ('feats', FeatureUnion(
            [
                ('pair', pair_question_pipe),
                ('single', single_question_pipe)
            ]
        ))
    ]
)

# pre-process pipe
feature_transformation = Pipeline(
    [
        ('feats', FeatureUnion(
            [
                ('clean_text_features', clean_text_features),
                ('lemma_text_features', lemma_text_features)
            ]
        ))
    ]
)


In [15]:
%%time
X_train_transform = feature_transformation.transform(X_train) ## this takes a really long time

CPU times: user 1h 10min 40s, sys: 5min 58s, total: 1h 16min 39s
Wall time: 25min 12s


## Configure the search

In [52]:
skf = StratifiedKFold(n_splits=3, random_state=42)

# fixed params
rf_params = {
    'n_jobs': 4,
    'random_state': 42,
    'verbose': 1
}

# tuning parameters -- start with estimators as I know 500 gives a very good AUC
rf_search_params = {
    'n_estimators': Integer(500, 750)
}

bayes_params = {
    'estimator': RandomForestClassifier(**rf_params),
    'scoring': 'roc_auc',
    'search_spaces': rf_search_params,
    'n_iter': 10,
    'cv': skf,
    'n_jobs': 1,
    'random_state': 42,
    'verbose': 1
}

search_cv = BayesSearchCV(**bayes_params)

In [53]:
def progress(optim_results):
    print(f'Best AUC: {search_cv_results.best_score_:.6f}')

In [54]:
%%time
search_cv_results = search_cv.fit(X_train_transform, y_train, callback=progress)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   20.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done 603 out of 603 | elapsed:  4.5min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    4.4s
[Parallel(n_jobs=4)]: Done 603 out of 603 | elapsed:    6.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   20.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  3.3min


Best AUC: 0.772373
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   20.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  3.8min
[Parallel(n_jobs=4)]: Done 709 out of 709 | elapsed:  6.2min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    4.4s
[Parallel(n_jobs=4)]: Done 709 out of 709 | elapsed:    7.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   20.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done 709 out of 709 | elapsed:  5.3min finished
[Parallel(n

Best AUC: 0.772373
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Best AUC: 0.772373
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   16.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done 654 out of 654 | elapsed:  4.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done 654 out of 654 | elapsed:    5.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   16.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  2.8min
[Parallel(n_jobs=4)]: Done 654 out of 654 | elapsed:  4.2min finished
[Parallel(n

Best AUC: 0.772373
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   16.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  2.8min
[Parallel(n_jobs=4)]: Done 636 out of 636 | elapsed:  4.1min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done 636 out of 636 | elapsed:    5.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   16.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done 636 out of 636 | elapsed:  4.1min finished
[Parallel(n

KeyboardInterrupt: 

In [31]:
pd.DataFrame(search_cv_results.cv_results_).sort_values('mean_test_score', ascending=False)

,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_n_estimators,params
2,0.807296,0.804316,0.808667,0.806759,0.001816,1,131.529183,12.478978,2.250522,0.141808,gini,6,552,"{'criterion': 'gini', 'max_depth': 6, 'n_estim..."
1,0.806255,0.803568,0.807976,0.805933,0.001814,1,208.998184,2.855084,2.383416,0.046274,entropy,6,652,"{'criterion': 'entropy', 'max_depth': 6, 'n_es..."
5,0.806177,0.803546,0.807995,0.805906,0.001827,1,193.943244,14.227560,2.188284,0.129094,entropy,6,582,"{'criterion': 'entropy', 'max_depth': 6, 'n_es..."
7,0.806156,0.803565,0.807893,0.805871,0.001779,1,258.290231,25.665571,2.720410,0.047840,entropy,6,748,"{'criterion': 'entropy', 'max_depth': 6, 'n_es..."
0,0.800919,0.798106,0.802275,0.800434,0.001736,1,177.414965,1.722789,3.251382,0.082671,gini,5,966,"{'criterion': 'gini', 'max_depth': 5, 'n_estim..."
9,0.800842,0.798036,0.802262,0.800380,0.001756,1,171.988086,8.928521,2.989161,0.132966,gini,5,871,"{'criterion': 'gini', 'max_depth': 5, 'n_estim..."
8,0.799891,0.797249,0.801186,0.799442,0.001638,1,292.883407,3.443255,3.663260,0.156456,entropy,5,936,"{'criterion': 'entropy', 'max_depth': 5, 'n_es..."
6,0.799905,0.797208,0.800985,0.799366,0.001588,1,187.357691,6.478847,2.555201,0.426496,entropy,5,680,"{'criterion': 'entropy', 'max_depth': 5, 'n_es..."
4,0.792160,0.789164,0.793501,0.791608,0.001813,1,195.669708,7.524297,2.623955,0.174881,entropy,4,763,"{'criterion': 'entropy', 'max_depth': 4, 'n_es..."
3,0.784138,0.780812,0.785160,0.783370,0.001857,1,144.786910,11.266310,2.386551,0.373208,entropy,3,799,"{'criterion': 'entropy', 'max_depth': 3, 'n_es..."


In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_t, X_v, y_t, y_v = train_test_split(X_train_transform, y_train, stratify=y_train, random_state=42, test_size = 0.33)

In [36]:
rf = RandomForestClassifier(n_estimators=500, n_jobs=4, random_state=42, verbose=1)

In [37]:
rf.fit(X_t, y_t)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   18.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:  3.5min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=4,
            oob_score=False, random_state=42, verbose=1, warm_start=False)

In [38]:
y_v_probs = rf.predict_proba(X_v)[:, 1]

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    4.4s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    4.9s finished


In [39]:
from sklearn import metrics

In [40]:
metrics.roc_auc_score(y_v, y_v_probs)

0.8689592428199138